# cf-python Regridding

* **Based on the ESMPy interface to the Earth System Modeling Framework (ESMF) library**
* **Coordinate systems: Spherical (regrids) or Cartesian (regridc)**
* **Regridding methods:**
    * **First order conservative** - conserves integral of field with respect to area
    * **Bilinear** - linear interpolation in 1-3 dimensions (2D for sperical coordinates)
    * **Higher order patch recovery** - polynomial based so better values/derivatives
    * **Nearest source to destination** - useful for integer fields such as land use
    * **Nearest destination to source** - multiple source points can contribute to one destination point
* **Global or regional source/destination grids in any combination**
* **Handles grids with 2D latitudes and longitudes including curvilinear, rotated pole and some tripolar grids**
* **Handles masking of both source and destination grids**

## Spherical regridding (regrids)

### Conservative regridding

In [ ]:
# Inline images in IPython Notebook - not needed in Python
%matplotlib inline

# Turn off warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Import the cf and cfplot packages
import cf
import cfplot as cfp

In [ ]:
# Read in ncas_data/precip_2010.nc and inspect the field
f = cf.read('ncas_data/precip_2010.nc')[0]
print f

In [ ]:
# Read in ncas_data/model_precip_DJF_means_low_res.nc and inspect the field
g = cf.read('ncas_data/model_precip_DJF_means_low_res.nc')[0]
print g

In [ ]:
# Regrid the first field to the grid of the second and inspect the results
h = f.regrids(g, method='conservative')
print h

In [ ]:
# Plot before and after
cfp.gopen(rows=1, columns=2)
cfp.gpos(1)
cfp.con(f[0], blockfill=True, lines=False, colorbar_label_skip=2)
cfp.gpos(2)
cfp.con(h[0], blockfill=True, lines=False, colorbar_label_skip=2)
cfp.gclose()

### Regridding from a tripolar grid

![alt text](https://www.gfdl.noaa.gov/wp-content/uploads/pix/user_images/mw/bipolar.gif "Tripolar Grid")

In [ ]:
# Read in ncas_data/tripolar.nc and select and inspect a field
f = cf.read('ncas_data/tripolar.nc')[0]
print f

In [ ]:
# Read in ncas_data/model_precip_DJF_means.nc and inspect the field
g = cf.read('ncas_data/model_precip_DJF_means.nc')[0]
print g

In [ ]:
# Regrid the field on the tripolar grid to the regular lat-long grid
h = f.regrids(g, method='bilinear', src_axes={'X': 'ncdim%x', 'Y': 'ncdim%y'}, src_cyclic=True)
print h

In [ ]:
# Plot the regridded data
cfp.levs(min=-1.0, max=1.2, step=0.2)
cfp.con(h)

### Higher order patch recovery versus bilinear regridding

In [ ]:
# Read in ncas_data/data5.nc and inspect the field
f = cf.read('ncas_data/data5.nc')[0].subspace[0, 0]
print f

In [ ]:
cfp.levs()
cfp.con(f)

In [ ]:
# Read in ncas_data/model_precip_DJF_means_low_res.nc and inspect the field
g = cf.read('ncas_data/model_precip_DJF_means_low_res.nc')[0]
print g

In [ ]:
# Regrid the first field to the grid of the second using bilinear interpolation
h = f.regrids(g, method='bilinear')
print h

In [ ]:
# Regrid the first field to the grid of the second using higher order patch recovery
i = f.regrids(g, method='patch')
print i

In [ ]:
# Find the y derivatives of the regridded fields
deriv_h = h.derivative('Y')
deriv_h.units = 'm.s-1.degrees-1'
deriv_i = i.derivative('Y')
deriv_i.units = 'm.s-1.degrees-1'

In [ ]:
# Plot the regridded fields and the differences between the derivatives
cfp.gopen(rows=1, columns=3)
cfp.gpos(1)
cfp.con(h, colorbar_label_skip=2, title='Bilinear')
cfp.gpos(2)
cfp.con(i, colorbar_label_skip=2, title='Higher order patch recovery')
cfp.gpos(3)
cfp.levs(min=-0.06, max=0.06, step=0.02)
cfp.cscale('scale1')
cfp.con(deriv_i - deriv_h, blockfill=True, lines=False, colorbar_label_skip=2, title='Difference of derivatives')
cfp.gclose()

### Regridding an integer field using the nearest neighbour method

In [ ]:
# Read in ncas_data/regions_low_res.nc and inspect the field
f = cf.read('ncas_data/regions.nc')[0]
print f

In [ ]:
# Read in ncas_data/model_precip_DJF_means_low_res.nc and inspect the field
g = cf.read('ncas_data/model_precip_DJF_means_low_res.nc')[0]
print g

In [ ]:
# Regrid regions to model grid using nearest source to destination regridding and inspect the result
h = f.regrids(g, method='nearest_stod')
print h

In [ ]:
# Plot before and after
cfp.gopen(rows=1, columns=2)
cfp.levs(min=1, max=10, step=1)
cfp.cscale()
cfp.gpos(1)
cfp.con(f, blockfill=True, lines=False)
cfp.gpos(2)
cfp.con(h, blockfill=True, lines=False)
cfp.gclose()

### Regridding with constructed coordinates

In [ ]:
# Read in ncas_data/precip_2010.nc and inspect the field
f = cf.read('ncas_data/precip_2010.nc')[0]
print f

In [ ]:
# Create dimension coordinates for the destination grid
import numpy as np
lon = cf.DimensionCoordinate(data=cf.Data(np.arange(-33, 64, 2.0), 'degrees_east'))
lat = cf.DimensionCoordinate(data=cf.Data(np.arange(-90, 91, 2.0), 'degrees_north'))

In [ ]:
# Create Voronoi bounds for the new dimension coordinates
lon.get_bounds(create=True, insert=True)
lat.get_bounds(create=True, insert=True)

In [ ]:
# Regrid the field to the grid of the new coordinates bilinearly and inspect the resulting field
g = f.regrids({'longitude': lon, 'latitude': lat}, method='conservative')
print g

In [ ]:
# Plot before and after
cfp.gopen(rows=1, columns=2)
cfp.levs()
cfp.gpos(1)
cfp.con(f[0], blockfill=True, lines=False, colorbar_label_skip=2)
cfp.gpos(2)
cfp.con(g[0], blockfill=True, lines=False, colorbar_label_skip=2)
cfp.gclose()

## Cartesian regridding (regridc)

### Regridding a time series

In [ ]:
# Read in ncas_data/precip_1D_yearly.nc and inspect the field
f = cf.read('ncas_data/precip_1D_yearly.nc')[0]
print f

In [ ]:
# Read in ncas_data/precip_1D_monthly.nc and inspect the field
g = cf.read('ncas_data/precip_1D_monthly.nc')[0]
print g

In [ ]:
# Regrid the first field to the grid of the second linearly and summarize the resulting field
h = f.regridc(g, axes='T', method='bilinear')
print h

In [ ]:
# Plot before and after
cfp.gopen(rows=1, columns=2)
cfp.gpos(1)
cfp.lineplot(f, marker='o', color='red')
cfp.gpos(2)
cfp.lineplot(h, marker='o', color='blue')
cfp.gclose()

### 2D cartesian regridding

In [ ]:
# Read in ncas_data/u_216.nc and inspect the field
f = cf.read('ncas_data/u_n216.nc')[0]
print f

In [ ]:
# Read in ncas_data/u_96.nc and inspect the field
g = cf.read('ncas_data/u_n96.nc')[0]
print g

In [ ]:
# Save the pressure corodinates and their keys
p_src = f.dim('Z').copy()
src_key = f.dim('Z', key=True)
p_dst = g.dim('Z').copy()
dst_key = g.dim('Z', key=True)

In [ ]:
# Take the log of the pressures
f.dim('Z').log(base=10, i=True)
g.dim('Z').log(base=10, i=True)

In [ ]:
# Regrid the source field and inspect the result
h = f.regridc(g, axes=('Y', 'Z'), method='bilinear')
print h

In [ ]:
# Insert the saved destination pressure coordinate into the regridded field
key = h.dim('Z', key=True)
h.insert_dim(p_dst, key=key)
print h

In [ ]:
# Reinsert the saved pressure coordinates into the original fields
f.insert_dim(p_src, key=src_key)
g.insert_dim(p_dst, key=dst_key)

In [ ]:
# Plot before and after
cfp.con(f, title='n216', ylog=True)
cfp.con(g, title='regridded', ylog=True)